In [54]:
# These are standard modules
import time
import numpy as np
from scipy import linalg
import sympy as sym
import matplotlib.pyplot as plt
from IPython.display import display, Latex, Markdown
from sympy.physics import mechanics


# Suppress the use of scientific notation when printing small numbers
np.set_printoptions(suppress=True)

In [113]:
# Define a function to return a state-space model for a given choice of equilibrium point.
t = sym.symbols('t')
J_1z = sym.symbols('J_1z')
J_2x, J_2z = sym.symbols('J_2x, J_2z')
J_3x, J_3y, J_3z = sym.symbols('J_3x, J_3y, J_3z')
r = sym.symbols('r')
m = sym.symbols('m')
g = sym.symbols('g')
q1, q2 = mechanics.dynamicsymbols('q1, q2')
v1, v2 = sym.symbols('v1, v2') #v = qdot
v1 = q1.diff(t)
v2 = q2.diff(t)
w1, w2 = sym.symbols('w1, w2') #qdd is q double dot
w1 = v1.diff(t)
w2 = v2.diff(t)
a1, a2, a3, a4, a5, a6, a7, a8 = sym.symbols('a1, a2, a3, a4, a5, a6, a7, a8')
tau = sym.symbols('tau') 
J_1z = 0.5
J_2x = 0.001
J_2z = 0.001
J_3x = 0.01
J_3y = 0.01
J_3z = 0.01
m = 1.0
r = 2.0
g = 9.81
a1 = -J_3y + 2 * J_3z
a2 = 2* J_3y 
a3 = -2 * g * m * r
a4 = 2 * J_1z + 2* J_2z + 2 * m * r**2
a5 = 2* J_3z
a6 = (J_3y - J_3z)/(2 * (J_2x + J_3x))
a7 = -J_3y/(J_2x + J_3x)
a8 = 1/(J_2x + J_3x)
vrot = 1000
q1e = np.pi
v1e = 0
q2e = 0
v2e = 0
taue = 0

def get_model(q1e, v1e, q2e, v2e, tau):
    f = sym.Matrix([
        v1, 
        (((a1 * sym.sin(2 * q2) * v1 * v2) + (a2 * sym.cos(q2) * v2*1000) + (a3)*sym.sin(q1)) / (a4 + (a5 * sym.cos(q2)**2))),
        v2,
        a6 * sym.sin(2 * q2) * v1**2 + a7 * sym.cos(q2) * v1*1000 + a8 * tau
    ])
    f = sym.nsimplify(f, rational=True)
    f_num = sym.lambdify([q1, v1, q2, v2, tau], f)
    if not np.allclose(f_num(q1e, v1e, q2e, v2e, taue), 0.):
        raise Exception('equilibrium point is invalid')
    
    A_sym = f.jacobian([q1, v1, q2, v2])
    B_sym = f.jacobian([tau])
    A_num = sym.lambdify([q1, v1, q2, v2, tau], A_sym)
    B_num = sym.lambdify([q1, v1, q2, v2, tau], B_sym)
    A = A_num(q1e, q2e, v1e, v2e, taue).astype(float)
    B = B_num(q1e, q2e, v1e, v2e, taue).astype(float)
    return A, B
A = get_model(q1e, v1e, q2e, v2e, tau)[0]
B = get_model(q1e, v1e, q2e, v2e, tau)[1]

A, B

(array([[   0.        ,    1.        ,    0.        ,    0.        ],
        [   4.34936821,    0.        ,    0.        ,    2.21680337],
        [   0.        ,    0.        ,    0.        ,    1.        ],
        [   0.        , -909.09090909,    0.        ,    0.        ]]),
 array([[ 0.        ],
        [ 0.        ],
        [ 0.        ],
        [90.90909091]]))

In [ ]:
## Find K
import numpy as np

K = np.zeros(4)

# F  = A - B@K
# s = linalg.eigvals(F)
# if (s.real < 0).all() and (s.imag == 0).all():

count = 0
while count == 0:
    for i in range(4):
        K[i] = np.random.rand()
    F = A - B.T@K
    s = linalg.eigvals(F)
    if (s.real < 0).all():
        count += 1
K, s

In [34]:
class Controller:
    def __init__(self, K, q1e, v1e, q2e, v2e, taue):
        self.K = K
        self.q1e = q1e
        self.q2e = q2e
        self.v1e = v1e
        self.v2e = v2e
        self.taue = taue
    
    def reset(self):
        pass
    
    def run(
            self,
            t,
            platform_angle,
            platform_velocity,
            gimbal_angle,
            gimbal_velocity,
        ):
        
        # Find state
        x = np.array([
            platform_angle - self.q1e,
            platform_velocity - self.v1e,
            gimbal_angle - self.q2e,
            gimbal_velocity - self.v2e,
        ])
        
        # Find input
        u = - self.K @ x
        
        # Find actuator command
        tau = u[0]
        
        gimbal_torque = -tau
        return gimbal_torque
    
    def get_data(simulator, q1e, v1e, q2e, v2e, taue, K, q1i, v1i, q2i, v2i, max_time=5.0):
    # Create controller
        controller = Controller(K, q1e, v1e, q2e, v2e, taue)
    
    # Reset simulator
        simulator.reset(
            platform_angle=q1i,
            platform_velocity=v1i,
            gimbal_angle=q2i,
            gimbal_velocity=v2i,
        )

    # Reset controller
        controller.reset()

    # Run simulator
        data = simulator.run(
            controller,
            max_time=max_time,
        )
    
    # Return data
        return data
    
def get_x_numeric(data, q1e, v1e, q2e, v2e, A, B):
    # Define initial conditions
    x0 = np.array([
        data['platform_angle'][0] - q1e,
        data['platform_velocity'][0] - v1e,
        data['gimbal_angle'][0] - q2e,
        data['gimbal_velocity'][0] - v2e,
    ])

    # Create empty array to hold state at each time
    x = []

    # Compute state at each time
    for t in data['t']:
        x.append(linalg.expm((A - B @ K) * t) @ x0)
    x = np.array(x)
    
    # Return state at each time
    return x

def get_x_symbolic(A, B, K):
    # Model
    A = sym.nsimplify(sym.Matrix(A), rational=True)
    B = sym.nsimplify(sym.Matrix(B), rational=True)
    K = sym.nsimplify(sym.Matrix(K), rational=True)
    
    # Variables
    t, x_1i, x_2i, x_3i, x_4i = sym.symbols('t, x_1i, x_2i, x_3i, x_4i', real=True)
    
    # Initial condition
    x0 = sym.Matrix([x_1i, x_2i, x_3i, x_4i])
    
    # Solution
    x = sym.exp((A - B @ K) * t) @ x0
    return x

def show_x_symbolic(A, B, K):
    x = get_x_symbolic(A, B, K)
    print('\nSOLUTION TO CLOSED-LOOP SYSTEM')
    display(Markdown(f'$$x(t) = {sym.latex(x)}$$'))
    
# def show_results(data, q1e, v1e, q2e, v2e, taue, A, B, K, q1i, v1i, v2i, show_pred=False):
#     t = data['t']
#     q1 = data['platform_angle']
#     v1 = data['platform_velocity']
#     q2 = data['gimbal_angle']
#     v2 = data['gimbal_velocity']
#     x1 = q1 - q1e
#     x2 = v1 - v1e
#     x3 = q2 - q2e
#     x4 = v2 - v2e
    
#     fig, ((ax_q1, ax_x1), (ax_v1, ax_x2), (ax_q2, ax_x3), (ax_v2, ax_x4)) = plt.subplots(4, 2, figsize=(15, 12), sharex=True)

#     ax_q1.plot(t, q1, label='$q_1$', linewidth=4)
#     ax_q1.plot(t, np.ones_like(t) * q1e, '--', label='$q_1$ (desired)', linewidth=4)
    
#     ax_v1.plot(t, v1, label='$v_1$', linewidth=4)
#     ax_v1.plot(t, np.ones_like(t) * v1e, '--', label='$v_1$ (desired)', linewidth=4)
    
#     ax_v2.plot(t, v2, label='$v_2$', linewidth=4)
#     ax_v2.plot(t, np.ones_like(t) * v2e, '--', label='$v_2$ (desired)', linewidth=4)
    
#     ax_x1.plot(t, x1, label='$x_1$', linewidth=4)
#     ax_x1.plot(t, np.zeros_like(t), '--', label='$x_1$ (desired)', linewidth=4)
    
#     ax_x2.plot(t, x2, label='$x_2$', linewidth=4)
#     ax_x2.plot(t, np.zeros_like(t), '--', label='$x_2$ (desired)', linewidth=4)
    
#     ax_x3.plot(t, x3, label='$x_3$', linewidth=4)
#     ax_x3.plot(t, np.zeros_like(t), '--', label='$x_3$ (desired)', linewidth=4)
    
#     if show_pred:
#         # Get x(t) and extract components
#         x_num = get_x_numeric(data, q1e, v1e, v2e, A, B)
#         x1_num = x_num[:, 0]
#         x2_num = x_num[:, 1]
#         x3_num = x_num[:, 2]
        
#         # Compute q1(t), v1(t), and v2(t) from x(t)
#         q1_num = x1_num + q1e
#         v1_num = x2_num + v1e
#         v2_num = x3_num + v2e
        
#         # Add everything to plots
#         ax_q1.plot(t, q1_num, ':', label='$q_1$ (linear)', linewidth=6, color='C3')
#         ax_v1.plot(t, v1_num, ':', label='$v_1$ (linear)', linewidth=6, color='C3')
#         ax_v2.plot(t, v2_num, ':', label='$v_2$ (linear)', linewidth=6, color='C3')
#         ax_x1.plot(t, x1_num, ':', label='$x_1$ (linear)', linewidth=6, color='C3')
#         ax_x2.plot(t, x2_num, ':', label='$x_2$ (linear)', linewidth=6, color='C3')
#         ax_x3.plot(t, x3_num, ':', label='$x_3$ (linear)', linewidth=6, color='C3')
        
#     ax_q1.grid()
#     ax_q1.legend(fontsize=16)
#     ax_q1.tick_params(labelsize=14)
#     ax_q1.set_ylim(np.pi - 0.5, np.pi + 0.5)
    
#     ax_v1.grid()
#     ax_v1.legend(fontsize=16)
#     ax_v1.tick_params(labelsize=14)
#     ax_v1.set_ylim(-1, 1)
    
#     ax_v2.grid()
#     ax_v2.legend(fontsize=16)
#     ax_v2.tick_params(labelsize=14)
#     ax_v2.set_ylim(-5, 5)

#     ax_x1.grid()
#     ax_x1.legend(fontsize=16)
#     ax_x1.tick_params(labelsize=14)
#     ax_x1.set_ylim(-0.5, 0.5)
        
#     ax_x2.grid()
#     ax_x2.legend(fontsize=16)
#     ax_x2.tick_params(labelsize=14)
#     ax_x2.set_ylim(-1, 1)
    
#     ax_x3.grid()
#     ax_x3.legend(fontsize=16)
#     ax_x3.tick_params(labelsize=14)
#     ax_x3.set_ylim(-5, 5)
    
#     ax_v2.set_xlabel('time (s)', fontsize=20)
#     ax_v2.set_xlim([data['t'][0], data['t'][-1]])
#     ax_x3.set_xlabel('time (s)', fontsize=20)
#     ax_x3.set_xlim([data['t'][0], data['t'][-1]])
    
#     fig.tight_layout()
#     plt.show()